# SBO Markings Counts (2020)
This notebook will demonstrate how to access attribute table from shapefiles in order to create a markings asset counts for each street segment

<div style="text-align:center"><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/4/47/Road_resurfacing.jpg/1280px-Road_resurfacing.jpg'></div>

## Introduction
The purpose of this notebook is to estimate the total pavement markings asset counts over the next 5 years from ficscal year 2020 to 2024 using the markings asset feature layers from ArcGIS Online.

<i><b>Disclaimer:</b> This product is for informational purposes and may not have been prepared for or be suitable for legal, engineering, or surveying purposes. No warranty is made by the City of Austin regarding specific accuracy or completeness.</i>

## Imports
The packages used for this project are:
- [pandas](https://pandas.pydata.org/) to create dataframe of extracted table and transform the data
- [geopandas](http://geopandas.org/mapping.html) to access attribute table of 5 year selected segments for SBO
- [arcgis](https://esri.github.io/arcgis-python-api/apidoc/html/) to search for markings feature layer dataset

In [1]:
import pandas as pd
import geopandas

from arcgis.gis import GIS
from arcgis.features import FeatureLayer

%run C:\Users\Govs\Projects/FeatureLayerDataFrame.py

## Constants


In [2]:
gis = GIS("https://austin.maps.arcgis.com/home/index.html", client_id='CrnxPfTcm7Y7ZGl7')

Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://austin.maps.arcgis.com/sharing/rest/oauth2/authorize?client_id=CrnxPfTcm7Y7ZGl7&response_type=code&expiration=-1&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob


Enter code obtained on signing in using SAML:  ············································································································································································································································


In [3]:
FOLDER = r'G:\ATD\Signs_and_Markings\MARKINGS\Whereabouts WORK ORDERS\SBO_2020-2024_Service_Plan'
SHP = FOLDER + r'\SBO_FiveYear_Service_Plan_09.shp'

## Import Dataset
Looking into the 2020 SBO Plans to extract Segment IDs and create list of segment IDs between bike projects and all streets

In [22]:
austin = geopandas.read_file(SHP).query('YR == 2020').filter(['SEGMENT_ID']).reset_index(drop=True)
segments = list(austin.SEGMENT_ID)
# sql no filter for existing facility on bikes variable instead of "" blank parameter
# "AND EXISTING_FACILITY LIKE 'BIKELN%'"
bikes = FeatureLayerDataFrame('bicycle_facilities').query_segments('STREET_SEGMENT_ID',segments,"")
bike_segments = list(bikes.STREET_SEGMENT_ID)

Here it will print the number of bike lane segments affected for 2020

In [25]:
print(len(segments))
#print(len(bike_segments))
bike_df = pd.DataFrame(bike_segments)
#print(bike_segments)
#writer = pd.ExcelWriter(FOLDER +r'\bike_segments.xlsx', engine='xlsxwriter')
#bike_df.to_excel(FOLDER +r'\bike_segments.xlsx', sheet_name = 'Markings Counts')

3095


## Group and Append Datasets
For specialty markings and short line, the sum is calculated by counting the number of rows grouped by markings type. For longline, the sum is calculated by adding the total linear miles where longline exists.

In [144]:
# Method to select segments
def select_segments(seg_list,col_name):
    sp = FeatureLayerDataFrame('markings_specialty_point').query_segments('SEGMENT_ID',seg_list,'')
    sl = FeatureLayerDataFrame('markings_short_line').query_segments('SEGMENT_ID',seg_list,'')
    sp_l = FeatureLayerDataFrame('markings_specialty_line').query_segments('SEGMENT_ID',seg_list,'')
    lines = FeatureLayerDataFrame('atd_maintained_streets').query_segments('SEGMENT_ID',seg_list,'')
    sp[col_name] = sl[col_name] = sp_l[col_name] = 1
    
    # Variables
    repl = ['CROSSWALK','SCHOOL_ZONE_LINE','STOP_LINE','YIELD_LINE','Y','CURBS','DELINEATORS']
    val = ['Crosswalk','School Zone Line','Stop Line','Yield Line', 'Longline Miles','Curbs','Delineators']

    # Data Analysis
    sp_counts = sp.groupby('SPECIALTY_POINT_TYPE').count()[[col_name]].reset_index(drop=False).rename(
        columns={'SPECIALTY_POINT_TYPE':'MARKINGS TYPE'})
    sl_counts = sl.groupby('SHORT_LINE_TYPE').count()[[col_name]].reset_index(drop=False).rename(
        columns={'SHORT_LINE_TYPE':'MARKINGS TYPE'})
    sp_l_counts = sp_l.groupby('SPECIALTY_LINE_TYPE').count()[[col_name]].reset_index(drop=False).rename(
        columns={'SPECIALTY_LINE_TYPE':'MARKINGS TYPE'})
    lines_counts = lines.groupby('LONGLINE').sum()[['LINEAR_MILES']].reset_index(drop=False).query("LONGLINE == 'Y'").rename(
        columns={'LONGLINE':'MARKINGS TYPE','LINEAR_MILES':col_name})

    # Appending
    df = lines_counts.append([sp_counts,sl_counts,sp_l_counts],sort=True).replace(repl,val).set_index('MARKINGS TYPE')
    df[col_name] = df[col_name].apply(int)
    return df

Here we create two columns. One including all streets, and another that only includes bike lanes.

In [145]:
df = select_segments(segments,'Sum of All')
df_bikes = select_segments(bike_segments,'Sum of Bike Lane')

## Display Markings Count Table
The final dataset will include sum of all, sum of bike lanes, and sum without bike lanes. This is to notify Active Transportation of any operations from SBO that would require them to make plans

In [151]:
df_final = df.merge(df_bikes,on='MARKINGS TYPE')
df_final['Sum without Bike Lanes'] = df_final["Sum of All"] - df_final["Sum of Bike Lane"]

In [152]:
display(df_final)

,Sum of All,Sum of Bike Lane,Sum without Bike Lanes
MARKINGS TYPE,,,
Longline Miles,140,17,123
Bicyclist symbol,319,190,129
Bike arrow,662,384,278
Chevron symbol,239,88,151
Diagonal crosshatch,522,313,209
Green launch pad,10,1,9
Green pad,157,37,120
Lane reduction arrow,5,5,0
Left arrow,318,101,217


## Export to Excel

In [153]:
writer = pd.ExcelWriter(FOLDER +r'\2020_Markings_Counts.xlsx', engine='xlsxwriter')
df_final.to_excel(writer, sheet_name = 'Markings Counts')
wb = writer.book
ws = writer.sheets['Markings Counts']
ws.set_column('A:D',29)
writer.save()